In [21]:
import pandas as pd
import importlib
import berechnen_wp, pv_profil, lastprofile_VDI4655, temperatur_aussen, try_region, heizkurve
importlib.reload(berechnen_wp)

h, w, twe, s = lastprofile_VDI4655.get_jahresenergiebedarf('1984 - 1994', 200, 3, 0)
TRY_region, T_n_aussen = try_region.get_try_t_n_aussen(40599)
df = lastprofile_VDI4655.get_lastprofile(w, s, twe, 200, TRY_region, 3)
df['T_aussen'] = temperatur_aussen.get_hourly_temperature(40599, 2014)
hz, T_soll, T_n_vor, T_n_rueck = heizkurve.get_heizkurve('Heizkörper', df['T_aussen'], T_n_aussen)
df['T_vor'] = hz['T_vor']
df['T_rueck'] = hz['T_rueck']
wp_groesse = berechnen_wp.get_waermepumpe(h)
heizleistung_auslegung = heizkurve.get_heizleistung(T_n_aussen, h, T_soll)
df = heizkurve.get_heizleistung_profil(df, heizleistung_auslegung)
df = heizkurve.get_cop(wp_groesse, df)
V_ps, PS_verlust, Q_ps = berechnen_wp.get_pufferspeicher(h, T_n_vor, T_n_rueck)
df, P_el, COP = berechnen_wp.ohne_pv(df, Q_ps, PS_verlust)
pv = pv_profil.get_pv_profil(40599, 2014, 10)
df = berechnen_wp.mit_pvbs(df, pv, 10, 11)
df

The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.




,Strombedarf,Warmwasserbedarf,Heizwärmebedarf,T_aussen,T_vor,T_rueck,Heizleistung,COP,Wärmegehalt PS,Ladezustand PS,...,elekt. Leistungaufnahme,therm. Entnahmelesitung,PV Ertrag,battery_soc,battery_charge,battery_discharge,überschuss,netzbezug,eigenverbrauch,PV to WP
Zeit,,,,,,,,,,,,,,,,,,,,,
2014-01-01 00:00:00,0.209688,0.000000,0.971386,6.3,43.881684,38.953627,5.418421,3.73,3.483000,1.000000,...,1.452660,3.965761,0.0,3.268387,0.0,1.731613,0.0,0.000000,0.0,0.0
2014-01-01 01:00:00,0.199006,0.000000,1.544818,5.0,45.056198,39.660515,6.228070,NaN,1.874848,0.538285,...,0.000000,6.228070,0.0,3.061089,0.0,0.207298,0.0,0.000000,0.0,0.0
2014-01-01 02:00:00,0.212510,0.000000,2.094290,5.1,44.967379,39.607667,6.165789,3.50,5.883014,1.000000,...,1.761654,4.404135,0.0,1.004668,0.0,2.056421,0.0,0.000000,0.0,0.0
2014-01-01 03:00:00,0.209700,0.000000,2.548971,5.5,44.609629,39.393802,5.916667,NaN,3.270709,0.939050,...,0.000000,5.916667,0.0,1.000000,0.0,0.004668,0.0,0.205219,0.0,0.0
2014-01-01 04:00:00,0.205412,0.000000,2.401328,5.7,44.429232,39.285347,5.792105,NaN,0.806048,0.231423,...,0.000000,5.792105,0.0,1.000000,0.0,0.000000,0.0,0.205412,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-12-31 19:00:00,0.744177,0.041829,9.012746,1.9,47.700356,41.189565,8.158772,2.96,0.000000,0.000000,...,3.044846,5.113925,0.0,1.000000,0.0,0.000000,0.0,3.789024,0.0,0.0
2014-12-31 20:00:00,0.721275,0.000000,7.224973,1.4,48.109164,41.418517,8.470175,2.86,1.181869,0.339325,...,2.961600,5.508576,0.0,1.000000,0.0,0.000000,0.0,3.682874,0.0,0.0
2014-12-31 21:00:00,0.695183,0.000000,3.627545,1.1,48.352353,41.553792,8.657018,2.77,6.148008,1.000000,...,3.125277,5.531740,0.0,1.000000,0.0,0.000000,0.0,3.820460,0.0,0.0


In [29]:
importlib.reload(berechnen_wp)
strompreis = 0.358
ergebnisse = berechnen_wp.ersparnis_bs(df, 10, strompreis)
berechnen_wp.print_ersparnis(ergebnisse)

Haushaltsstrombedarf in kWh:  4980
Wärmepumpe Strombedarf in kWh:  6634
Jahresertrag in kWh:  11282
Eigenverbrauch in kWh:  5303
Geladene PV-Strom in Batteriespeicher in kWh:  2098
Geladene PV-Strom in Wärmepumpe in kWh:  2135
Direkter Verbrauch PV-Strom in kWh:  3205

Netzbezug in kWh:  5920
Einspeisung ins Netz in kWh:  5421

Stromkosten ohne PV in €/a:  4157.82
Stromkosten mit PV & BS in €/a:  2119.36
Einspeisevergütung in €/a:  431.51
Stromkosten Einsparung in €/a:  2469.97
